In [ ]:
! pip install langchain
! sudo pip3 install beautifulsoup4
! pip install langchain-community
! pip install langchain-chroma
! pip install cohere
! pip install langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 679.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader

from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from langchain_community.llms import Ollama

llm = Ollama(model="llama3",base_url="https://b8c7-34-82-84-64.ngrok-free.app") #Change URL here without last backslash

#os.environ["COHERE_API_KEY"] = "z38tBCH9ChwAXYvSQLhgzceurRnupT5XhDMMiDgv"

from langchain_community.chat_models import ChatCohere



# Load, chunk and index the contents of the blog.
loader =WebBaseLoader(["https://www.nseindia.com/get-quotes/equity?symbol=RELIANCE","https://www.bseindia.com/stock-share-price/reliance-industries-ltd/reliance/500325/","https://en.wikipedia.org/wiki/Reliance_Industries","https://www.moneycontrol.com/news/tags/reliance-industries.html"])


docs = loader.load()
print(docs)
from langchain_community.embeddings import CohereEmbeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=CohereEmbeddings(user_agent="my-app",cohere_api_key="szIbFsIQCCVc3cE3jtHOabgGtrEDuTIQLucDHCFg"))# bhumika cohere api
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("Answer all questions: 1st) Who is reliance owner? 2nd) What is reliance current stock price? 3rd) Who founded Reliance?"))

[Document(metadata={'source': 'https://www.nseindia.com/get-quotes/equity?symbol=RELIANCE', 'title': '\r\n    Reliance Industries Limited Share Price Today, Stock Price, Live NSE News, Quotes, Tips – NSE India\r\n', 'description': 'Reliance Industries Limited Share Price Today, Live NSE Stock Price: Get the latest Reliance Industries Limited news, company updates, quotes, offers, annual financial reports, graph, volumes, 52 week high low, buy sell tips, balance sheet, historical charts, market performance, capitalisation, dividends, volume, profit and loss account, research, results and more details at NSE India.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\r\n    Reliance Industries Limited Share Price Today, Stock Price, Live NSE News, Quotes, Tips – NSE India\r\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOption Chain\nListings\nIPO\nCirculars\nDaily Reports\nHolidays\nPress Rel

<ipython-input-2-8d702599022e>:29: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereEmbeddings``.
  vectorstore = Chroma.from_documents(documents=splits, embedding=CohereEmbeddings(user_agent="my-app",cohere_api_key="szIbFsIQCCVc3cE3jtHOabgGtrEDuTIQLucDHCFg"))# bhumika cohere api


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Here are the answers to your questions based on the provided context:

1st) Who is reliance owner?
Mukesh Ambani owns 51% of Reliance Industries.

2nd) What is reliance current stock price?
The context does not provide the current stock price of Reliance Industries. I don't know.

3rd) Who founded Reliance?
Dhirubhai Ambani founded Reliance in 1958.
